In [21]:
import numpy as np, matplotlib.pyplot as plt, os

In [29]:
import scipy.io as sio

In [26]:
cwd = os.getcwd()

In [27]:
Pavia_scene_dir = os.path.join(cwd, "MAT Files", "Pavia.mat")

In [33]:
PaviaSc_gt_dir = os.path.join(cwd, "GT Files", "Pavia_gt.mat")

In [28]:
print(Pavia_scene_dir)

/Users/suhasmathey/Hyper_Spectral Imaging/Hyper-Spectral-Image-Analysis/MAT Files/Pavia.mat


In [34]:
PaviaSc_data_mat = sio.loadmat(Pavia_scene_dir)

In [44]:
PaviaSc_gt_mat = sio.loadmat(PaviaSc_gt_dir)

In [55]:
PaviaSc_gt_mat['pavia_gt']

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [56]:
PaviaSc_data_mat['pavia']

array([[[ 854,  601,  350, ..., 3664, 3636, 3643],
        [ 527,  642,  575, ..., 3834, 3725, 3768],
        [ 374,  322,  179, ..., 4318, 4311, 4321],
        ...,
        [ 367,  432,  461, ..., 2582, 2504, 2512],
        [ 261,  311,  366, ..., 2269, 2174, 2163],
        [1059,  678,  403, ..., 2245, 2135, 2136]],

       [[1060,  909,  596, ..., 2963, 2967, 2974],
        [ 707,  757,  646, ..., 3508, 3534, 3648],
        [ 143,  419,  417, ..., 4650, 4612, 4638],
        ...,
        [ 465,  547,  537, ..., 3156, 3052, 3035],
        [ 884,  615,  401, ..., 2792, 2667, 2639],
        [ 756,  401,  213, ..., 2600, 2484, 2445]],

       [[ 532,  545,  594, ..., 1675, 1653, 1680],
        [ 523,  491,  321, ..., 3339, 3349, 3403],
        [ 816,  681,  369, ..., 4627, 4600, 4650],
        ...,
        [ 408,  539,  436, ..., 3099, 3005, 3006],
        [ 393,  447,  476, ..., 3172, 3048, 3032],
        [ 798,  615,  489, ..., 3039, 2876, 2800]],

       ...,

       [[ 689,  560,  70

In [57]:
data = PaviaSc_data_mat['pavia']

In [59]:
gt_data = PaviaSc_gt_mat['pavia_gt']

In [61]:
indices = np.where(gt_data == 0)

In [62]:
print(indices)

(array([   0,    0,    0, ..., 1095, 1095, 1095]), array([  0,   1,   2, ..., 712, 713, 714]))
